In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
# Load the JSON data from the MPD slice file
with open('mpd.slice.0-999.json') as f:
    mpd_data = json.load(f)
# Extract playlists from the data
playlists = mpd_data["playlists"]

In [3]:
# Create a DataFrame from the playlist data
playlist_df = pd.DataFrame(playlists)

# Extract relevant features for clustering
X = playlist_df[['name', 'description']].fillna('').apply(lambda x: ' '.join(x), axis=1)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
X_vect = vectorizer.fit_transform(X)

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.cluster import MiniBatchKMeans
# k_values = range(1, 100)  # Try cluster numbers from 1 to 10
# inertia_values = []
# for k in k_values:
#     kmeans = MiniBatchKMeans(n_clusters=k, random_state=42)
#     kmeans.fit(X_vect)
#     inertia_values.append(kmeans.inertia_)
# plt.plot(k_values, inertia_values, marker='o')
# plt.xlabel('Number of clusters')
# plt.ylabel('Inertia')
# plt.title('Elbow Method for Optimal k')
# plt.show()

In [ ]:
# k_values = range(2, 100)
# best_silhouette_score = -1
# best_k = None
# for k in k_values:
#     minibatch_kmeans = MiniBatchKMeans(n_clusters=k, random_state=42)
#     cluster_labels = minibatch_kmeans.fit_predict(X_vect)
#     silhouette_avg = silhouette_score(X_vect, cluster_labels)
#     print(f"Number of clusters: {k}, Silhouette Score: {silhouette_avg}")
#     if silhouette_avg > best_silhouette_score:
#         best_silhouette_score = silhouette_avg
#         best_k = k

# print(f"Best number of clusters: {best_k}")

In [ ]:
#A general rule of thumb is that a silhouette score above 0.5 indicates a good clustering, 
#a silhouette score below 0.25 indicates a bad clustering,
#and a silhouette score between 0.25 and 0.5 indicates a fair clustering.

#40 clusters has a score of 0.5108932664557817, and seems to be a good elbow

In [5]:
with open('Vetting_Playlist.json', 'r') as file:
    # Read the contents
    json_data = file.read()
    # Parse the JSON
    vetting = json.loads(json_data)
    
playlist_data = {
    'name': [vetting['name']],
    'description': [vetting.get('description', '')]  # Assuming description may be missing
}

# Create a DataFrame from the playlist data
playlist_df = pd.DataFrame(playlist_data)

# Extract relevant features for clustering
X = playlist_df[['name', 'description']].fillna('').apply(lambda x: ' '.join(x), axis=1)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, stop_words='english')
V_vect = vectorizer.fit_transform(X)
print(V_vect)

  (0, 0)	1.0


In [6]:
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
minibatch_kmeans = MiniBatchKMeans(n_clusters=40, random_state=42)
cluster_labels = minibatch_kmeans.fit_predict(X_vect)
minibatch_kmeans.predict(V_vect)

ValueError: X has 1 features, but MiniBatchKMeans is expecting 157 features as input.

In [7]:
#chat gpt

# Load the JSON data from the MPD slice file
with open('mpd.slice.0-999.json') as f:
    mpd_data = json.load(f)

# Extract playlists from the data
playlists = mpd_data["playlists"]

# Create a DataFrame from the playlist data
playlist_df = pd.DataFrame(playlists)

# Extract relevant features for clustering
X = playlist_df[['name', 'description']].fillna('').apply(lambda x: ' '.join(x), axis=1)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words='english')
X_vect = vectorizer.fit_transform(X)

# Fit MiniBatchKMeans on the training data
minibatch_kmeans = MiniBatchKMeans(n_clusters=40, random_state=42)
cluster_labels = minibatch_kmeans.fit_predict(X_vect)

# Load the new playlist data
with open('Vetting_Playlist.json', 'r') as file:
    vetting_data = json.load(file)

# Create a DataFrame from the new playlist data
playlist_data = {
    'name': [vetting_data['name']],
    'description': [vetting_data.get('description', '')]  # Assuming description may be missing
}
playlist_df_new = pd.DataFrame(playlist_data)

# Extract relevant features for clustering from the new data
X_new = playlist_df_new[['name', 'description']].fillna('').apply(lambda x: ' '.join(x), axis=1)

# Vectorize the new data
V_vect = vectorizer.transform(X_new)

# Predict clusters for the new data
new_cluster_labels = minibatch_kmeans.predict(V_vect)
print(new_cluster_labels)


[32]


In [11]:
playlist_df_new = pd.Series(playlist_data)

In [8]:
group_32_playlists = playlist_df[cluster_labels == new_cluster_labels].reset_index(drop=True)
print(group_32_playlists)

                     name collaborative  pid  modified_at  num_tracks  \
0                     mat         false    3   1501027200         126   
1    I Put A Spell On You         false    6   1477094400          16   
2                     BOP         false    8   1508976000          46   
3                   abby          false   10   1509321600          72   
4                   SARAH         false   18   1507334400          68   
..                    ...           ...  ...          ...         ...   
398                 David          true  989   1508803200          33   
399             Michelle           true  990   1507420800          87   
400                   R&B         false  992   1508371200         126   
401                   🌈🌈🌈         false  993   1508112000          38   
402                  Daze         false  996   1479254400          17   

     num_albums  num_followers  \
0           107              1   
1            15              1   
2            37      

In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
playlist_series = pd.Series(playlist_data)
playlistDic = {
    'playlist_id': [],
    'songs': []
}
for i in range(len(group_32_playlists)):
    playlistDic['playlist_id'].append(i)
    playlistDic['songs'].append([])
    for y in range(len(group_32_playlists['tracks'][i])):
        playlistDic['songs'][i].append(group_32_playlists['tracks'][i][y]['track_name'])
        
        
playlists_df = pd.DataFrame(playlistDic)
        
# Convert the playlist series and DataFrame to strings
playlist_series_str = ' '.join(playlist_series[0])
playlists_df['songs_str'] = playlists_df['songs'].apply(lambda x: ' '.join(x))

# Create a CountVectorizer to convert playlists to vectors
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform([playlist_series_str] + list(playlists_df['songs_str']))

# Calculate cosine similarity
cosine_sim = cosine_similarity(X)[0][1:]

# Find the index of the most similar playlist
most_similar_index = cosine_sim.argmax()

# Get the most similar playlist ID
most_similar_playlist_id = playlists_df.iloc[most_similar_index]['playlist_id']
most_similar_playlist = playlists_df.iloc[most_similar_index]['songs']
songs_not_in_original = [song for song in most_similar_playlist if song not in playlist_series]

print(f"The most similar playlist ID is: {most_similar_playlist_id}")
print(f"The songs in the most similar playlist are: {most_similar_playlist}")
print(f"The songs in the most similar playlist not present in the original playlist are: {songs_not_in_original}")

The most similar playlist ID is: 0
The songs in the most similar playlist are: ['Danse macabre', 'Piano concerto No. 2 in G Minor, Op. 22: Piano concerto No. 2 in G Minor, Op. 22: II. Allegro scherzando', 'Dræm Girl', 'Sad Valentine', "You Say I'm in Love", 'Drowsy', 'Serving Goffman', 'Sea Song', 'Fortune Only', 'Looking out for You', 'Necromancer', 'Mistakes', 'Strangest Eyes', "It's Elizabeth", 'Pussy of my Dreams', '19', 'Neptune Estate', "Somebody's Talking", "Bent (Roi's Song)", 'Just Get High', 'I Love Seattle', 'Dark as Days', 'End It Now!', 'When Did Your Heart Go Missing?', 'Signs', 'Under A Rock', 'Two Weeks', 'Yet Again', 'Campus', 'Mess Me Around', 'Talk To Me', 'Trip Switch', 'Heart It Races - Dr Dog Version', 'Big Decisions', "It's All in Vain", 'Deadwater', 'C U Girl', 'HOME', 'CONTACTS', "Can't Come Down", 'Luv', 'Life Is What You Make It', 'Contraband', 'Miss You', "Money Won't Pay", "I'll Fall", "Nothing's Gonna Hurt You Baby", 'Affection', 'I Like You', 'Strange to 

In [ ]:
['Danse macabre', 'Piano concerto No. 2 in G Minor, Op. 22: Piano concerto No. 2 in G Minor, Op. 22: II. Allegro scherzando', 'Dræm Girl', 'Sad Valentine', "You Say I'm in Love", 'Drowsy', 'Serving Goffman', 'Sea Song', 'Fortune Only', 'Looking out for You', 'Necromancer', 'Mistakes', 'Strangest Eyes', "It's Elizabeth", 'Pussy of my Dreams', '19', 'Neptune Estate', "Somebody's Talking", "Bent (Roi's Song)", 'Just Get High', 'I Love Seattle', 'Dark as Days', 'End It Now!', 'When Did Your Heart Go Missing?', 'Signs', 'Under A Rock', 'Two Weeks', 'Yet Again', 'Campus', 'Mess Me Around', 'Talk To Me', 'Trip Switch', 'Heart It Races - Dr Dog Version', 'Big Decisions', "It's All in Vain", 'Deadwater', 'C U Girl', 'HOME', 'CONTACTS', "Can't Come Down", 'Luv', 'Life Is What You Make It', 'Contraband', 'Miss You', "Money Won't Pay", "I'll Fall", "Nothing's Gonna Hurt You Baby", 'Affection', 'I Like You', 'Strange to Hear', 'Sweet Emotion', 'Pink City', 'First Balloon To Nice', 'Rubdown', 'So Much Love So Little Time', 'Last Of The Good Old Days', 'Screaming', "Love's Lost Guarantee", '10:1', 'Crush The Camera', 'Every Moment', 'Endgame', 'El Scorcho', 'Veldt', 'Patroklos', 'suncream', 'Crying in Public', "It's Your Body 4", 'Poppies', 'Just Kiss Her', 'Infinity', 'Touch My Body', "We're Not Just Friends", 'Tried And True', 'Weak', 'Lucky Girl', 'Stay (Bedroom Tape)', 'Weird Science', 'No One Lives Forever', 'Wild Sex (In The Working Class) - 1988 Boingo Alive Version', 'Only A Lad', 'Private Life - Edited Version', 'i was all over her', "Hey Good Lookin'", 'All for Myself', "Words I Don't Remember", 'Lost Youth / Lost You', "What You Won't Do for Love", "Don't Want To Know If You Are Lonely", 'Eight Miles High', 'Turn It Around', "She's A Woman [And Now He Is A Man]", 'Crystal', 'Turning Japanese', 'Interference', 'Nothing Lasts', 'Nostalgic Feel', 'Spread A Little Sunshine', 'Drugs', 'Kalte Wut / Wenn Ich Einmal Reich Bin', 'Smoking the Day Away', 'High Plains Anthem', 'Who Got Da Props', 'Chief Rocka', "Love's Been Good To Me", 'Listen to the Warm', 'Fatalist Palmistry', 'So Sad, So Sad', 'Cult of Personality', 'Brazil', 'Bethlehem', 'Why iii Love The Moon.', "I'll Get Along", 'A Sunday Kind Of Love - Single Version', 'Chemistry', 'Soldiers Requiem', 'Camarilla', 'Vanilla Blue', 'Push', 'Driftless', "I'm So Ugly", 'Wounds', 'You Loved Me, You Killed Me', 'Embarrassingly Enough', 'Janitor', 'Elegy']
['Danse macabre', 'Piano concerto No. 2 in G Minor, Op. 22: Piano concerto No. 2 in G Minor, Op. 22: II. Allegro scherzando', 'Dræm Girl', 'Sad Valentine', "You Say I'm in Love", 'Drowsy', 'Serving Goffman', 'Sea Song', 'Fortune Only', 'Looking out for You', 'Necromancer', 'Mistakes', 'Strangest Eyes', "It's Elizabeth", 'Pussy of my Dreams', '19', 'Neptune Estate', "Somebody's Talking", "Bent (Roi's Song)", 'Just Get High', 'I Love Seattle', 'Dark as Days', 'End It Now!', 'When Did Your Heart Go Missing?', 'Signs', 'Under A Rock', 'Two Weeks', 'Yet Again', 'Campus', 'Mess Me Around', 'Talk To Me', 'Trip Switch', 'Heart It Races - Dr Dog Version', 'Big Decisions', "It's All in Vain", 'Deadwater', 'C U Girl', 'HOME', 'CONTACTS', "Can't Come Down", 'Luv', 'Life Is What You Make It', 'Contraband', 'Miss You', "Money Won't Pay", "I'll Fall", "Nothing's Gonna Hurt You Baby", 'Affection', 'I Like You', 'Strange to Hear', 'Sweet Emotion', 'Pink City', 'First Balloon To Nice', 'Rubdown', 'So Much Love So Little Time', 'Last Of The Good Old Days', 'Screaming', "Love's Lost Guarantee", '10:1', 'Crush The Camera', 'Every Moment', 'Endgame', 'El Scorcho', 'Veldt', 'Patroklos', 'suncream', 'Crying in Public', "It's Your Body 4", 'Poppies', 'Just Kiss Her', 'Infinity', 'Touch My Body', "We're Not Just Friends", 'Tried And True', 'Weak', 'Lucky Girl', 'Stay (Bedroom Tape)', 'Weird Science', 'No One Lives Forever', 'Wild Sex (In The Working Class) - 1988 Boingo Alive Version', 'Only A Lad', 'Private Life - Edited Version', 'i was all over her', "Hey Good Lookin'", 'All for Myself', "Words I Don't Remember", 'Lost Youth / Lost You', "What You Won't Do for Love", "Don't Want To Know If You Are Lonely", 'Eight Miles High', 'Turn It Around', "She's A Woman [And Now He Is A Man]", 'Crystal', 'Turning Japanese', 'Interference', 'Nothing Lasts', 'Nostalgic Feel', 'Spread A Little Sunshine', 'Drugs', 'Kalte Wut / Wenn Ich Einmal Reich Bin', 'Smoking the Day Away', 'High Plains Anthem', 'Who Got Da Props', 'Chief Rocka', "Love's Been Good To Me", 'Listen to the Warm', 'Fatalist Palmistry', 'So Sad, So Sad', 'Cult of Personality', 'Brazil', 'Bethlehem', 'Why iii Love The Moon.', "I'll Get Along", 'A Sunday Kind Of Love - Single Version', 'Chemistry', 'Soldiers Requiem', 'Camarilla', 'Vanilla Blue', 'Push', 'Driftless', "I'm So Ugly", 'Wounds', 'You Loved Me, You Killed Me', 'Embarrassingly Enough', 'Janitor', 'Elegy']